## Integração de dados

#### Masterlens, CASTLES

In [1]:
import pandas as pd
from astropy.coordinates import Angle
import sqlite3
import os

In [2]:
# dec: -27:52:25.7 ---> -27.873806 deg
def sexdeg_to_deg(coord):
    result = Angle(coord + " degrees").degree
    return result #*3600


In [3]:
# ra: 00h49m41.89s ---> 12.424542 deg
def hour_to_deg(coord):  # string coord
    result = Angle(coord).degree
    return result #*3600

In [4]:
# ra: 00:49:41.89 ---> 00h49m41.89s
def hour(coord):
    result = coord.replace(":", 'h', 1)
    result = result.replace(":", 'm', 1)
    result += 's'
    return result

In [5]:
def name_clean(element):
    name = element.replace('+', '-')
    name = name.replace('.', '-')
    name = name.split('-')[0]
    
    if 'PMNJ' in name:
        name = name.replace('PMNJ', 'PMN')
        
    elif name == 'QJ0158':
        name = 'CTQ414'
        
    elif name == 'LBQS1009':
        name = name.replace('LBQS', 'Q')
    
    return name

In [6]:
# returns df with castles data (with coordinates converted to degrees)
def castles_df():
        # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')

    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles['id'] = castles.index + 1 # from 1 to 100

    return castles

In [73]:
# returns df with castles data + imgs paths
def castles_imgs():
    
    castles_imgs = os.listdir('/home/viviane/GravLens/Database/Scraping/castles_imgs')
    names = []
    paths =[]
    for i in castles_imgs:
        name = i.rstrip('.gif')
        name = name.rstrip('H')
        name = name.rstrip('Hcc')
        name = name.rstrip('I')
        name = name.rstrip('Icc')
        name = name.rstrip('V')
        name = name.rstrip('Vcc')
        name = name.rstrip('K')
        name = name.rstrip('Kcc')
        name = name.rstrip('J')
        name = name.rstrip('Jcc')
        name = name.rstrip('R')
        name = name.rstrip('Rcc')
        
        names.append(name)
        path = i
        paths.append(path)

    images = {'name' : names, 'img_path' : paths }
    imgs_df = pd.DataFrame(images)
    
    # name - list of images
    imgs_df = imgs_df.groupby('name')['img_path'].apply(list)
    
    # castles df saved before in csv
    castles = castles_df()
    castles['name'] = castles['Lens Name'].apply(name_clean)
    castles.set_index(castles['name'], inplace=True)
    castles.drop('name', axis=1, inplace=True)
    
    # joining imgs df with system df
    result = pd.concat([castles, imgs_df], axis=1, join='outer', sort=False)
    result.set_index(castles['id'], inplace=True)
        
    return result

In [74]:
castles_imgs()

,Lens Name,zs,zl,RA (J2000),Dec (J2000),E(B-V),ms (mag),ml (mag),FGHz (mJy),Nim,"size ("")",dt (days),sigma (km/s),id,img_path
id,,,,,,,,,,,,,,,
1,Q0047-2808,3.60,0.48,12.424542,-27.873806,0.016,NaN,I=20.05,NaN,4ER,2.70,NaN,229±15,1,"[Q0047I.gif, Q0047Hcc.gif, Q0047V.gif, Q0047Ic..."
2,HE0047-1756,1.66,0.41,12.615958,-17.669111,0.022,I=16.53/2,I=18.97,NaN,2,1.44,NaN,NaN,2,"[HE0047I.gif, HE0047Icc.gif, HE0047V.gif, HE00..."
3,HST01247+0352,NaN,NaN,21.185000,3.866667,0.029,I=24.13/2,I=21.86,NaN,2,2.20,NaN,NaN,3,"[HST01247I.gif, HST01247Icc.gif, HST01247Hcc.g..."
4,HST01248+0351,NaN,NaN,21.190000,3.851667,0.029,NaN,NaN,NaN,2,0.74,NaN,NaN,4,NaN
5,B0128+437,3.124,NaN,22.805854,43.970317,0.082,NaN,NaN,F5=48,4,0.55,NaN,NaN,5,"[B0128Icc.gif, B0128I.gif]"
6,PMNJ0134-0931,2.216,0.77,23.648625,-9.517472,0.031,I=18.96/4,I=19.31,F5=529,5R,0.73,NaN,NaN,6,"[PMN0134V.gif, PMN0134Hcc.gif, PMN0134I.gif, P..."
7,Q0142-100,2.72,0.49,26.318750,-9.754722,0.031,I=16.47/2,I=18.72,F5~1,2,2.24,NaN,NaN,7,"[Q0142I.gif, Q0142H.gif, Q0142Vcc.gif, Q0142Hc..."
8,QJ0158-4325,1.29,0.317,29.672667,-43.417833,0.015,I=17.39/2,I=18.91,F8<0.2,2,1.22,NaN,NaN,8,"[CTQ414V.gif, CTQ414I.gif, CTQ414H.gif, CTQ414..."
9,B0218+357,0.96,0.68,35.272846,35.937161,0.068,I=19.28/2,I=20.06,F5=1209,2ER,0.34,10.5±0.4,NaN,9,"[B0218V.gif, B0218I.gif, B0218.gif, B0218Vcc.g..."


In [35]:
# importing masterlens data  -  n(50) lenses
# returns df with masterlens data from tables: system, coordinates(deg) and image paths

def masterlens_df(n):
    
    ml_files_path = '/home/viviane/GravLens/Database/Scraping/masterlens_results'
    ml_imgfolders_path = '/home/viviane/GravLens/Database/Scraping/masterlens_imgs'
    
    dfs=[]
    for i in range(1, n+1):
        
    # getting data from system file      
        ml = pd.read_csv(ml_files_path + '/system_{}.csv'.format(i))

            # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                        'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                        'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]


    # getting coords from coords file
        coords = pd.read_csv(ml_files_path + '/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']
    
    
    # getting image paths
        system_folder = ml_imgfolders_path + '/masterlens_{}_arquivos'.format(i)
        imgs_list = os.listdir(system_folder)
        imgs = [system_folder + img for img in imgs_list]
        ml['img_path'] =  str(imgs)
        
        
        ml['id'] = i + len(castles_df())
        dfs.append(ml)
        
    all_ml = pd.concat(dfs, sort=False)
    all_ml.set_index(all_ml['id'], inplace=True)
        
    return all_ml

In [36]:
masterlens_df(50)

,Name,Discovery,Discovery Date,Lens Kind,Lens Grade,Description,N Images,Einstein_R,Einstein_R quality,Stellar_v_disp,Stellar_v_disp_err,Ra(deg)_ml,Dec(deg)_ml,img_path,id,Alternate Names
id,,,,,,,,,,,,,,,,
101,SDSS J0008-0004,SLACS,2008-08-01,GAL-GAL,A,Luminous red galaxy lensing three background o...,3,1.160,SIE model,232.0,39.0,2.012310,-0.068960,['/home/viviane/GravLens/Database/Scraping/mas...,101,NaN
102,ACS J001423.02-302109.8,Serendipitous,2012-06-15,GAL-GAL,B,Arc,1,1.520,SIE model,NaN,NaN,3.595917,-30.352722,['/home/viviane/GravLens/Database/Scraping/mas...,102,NaN
103,ACS J001426.26-302255.9,Serendipitous,2012-06-15,GAL-GAL,A,Arc,1,1.000,SIE model,NaN,NaN,3.609417,-30.382194,['/home/viviane/GravLens/Database/Scraping/mas...,103,NaN
104,CSWA 21,CASSOWARY,2007-06,GAL-GAL,A,This system is the brightest Lyman Break Gala...,4,2.910,SIE model,397.0,55.0,5.670492,14.519565,['/home/viviane/GravLens/Database/Scraping/mas...,104,8 O'Clock Arc
105,SDSS J0029-0055,SLACS,2008-08-01,GAL-GAL,A,Luminous red galaxy lensing two background so...,2,0.960,SIE model,202.0,18.0,7.282400,-0.930710,['/home/viviane/GravLens/Database/Scraping/mas...,105,NaN
106,Abell 68,Serendipitous,2007-06-01,XRAY-CLUST,A,NaN,8,7.500,Reference redshift,NaN,NaN,9.271704,9.163992,['/home/viviane/GravLens/Database/Scraping/mas...,106,A68
107,SDSS J0044+0113,SLACS,2008-08,GAL-GAL,A,Comments on IFU+HST Data: Clear coincidence of...,2,0.790,SIE model,254.0,12.0,11.012080,1.220180,['/home/viviane/GravLens/Database/Scraping/mas...,107,NaN
108,Q0047-2808,Serendipitous,1996-01-01,GAL-QSO,A,Bright Einstein ring-like feature. The source ...,4,1.340,NaN,219.0,12.0,12.424747,-27.874020,['/home/viviane/GravLens/Database/Scraping/mas...,108,ER 0047-2808
109,HE0047-1756,HE survey,2004-05-01,GAL-QSO,A,The newly discovered double QSO HE 0047−1756 ...,2,0.751,SIE model,NaN,NaN,12.615500,-17.668890,['/home/viviane/GravLens/Database/Scraping/mas...,109,NaN


## Lens as a whole
### Merge  - Name

In [9]:
# system table ---------->  143 lenses
def system():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['Name', 'RA(deg)_c', 'Dec(deg)_c', 'E(B-V)_c', 'FGHz (mJy)_c', 'Nimages_c',
                     'size(arccsec)_c', 'sigma (km/s)_c']
    #castles = castles.set_index('Name')
    castles['id_c'] = castles.index
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):
        
        # getting data from system file
        ml = pd.read_csv('Scraping/masterlens_results/system_{}.csv'.format(i))

        # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                       'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                       'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]

        # getting coords from coords file
        coords = pd.read_csv('Scraping/masterlens_results/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    #all_ml = all_ml.set_index('Name')
    #all_ml['id_ml'] = range(100, 150)
    
    
    concat = pd.concat([castles, all_ml], sort=False)
    concat.reset_index(inplace=True, drop=True)
    concat.to_csv('concat.csv')
    # joining castles + ml data
    result = pd.merge(castles, all_ml, how ='outer', on='Name')  


    return result
        


### Merge - NACluster

In [12]:
# Preparing to run NACluster
def system_cluster():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = castles_df()
    castles = castles[['RA (J2000)', 'Dec (J2000)' ]]
    castles.columns = ['ra', 'dec']
    
    castles['idCatalog'] = 1
   
    
    # importing masterlens data  -  50 lenses
    ml = masterlens_df()
    ml = ml[['Ra(deg)_ml', 'Dec(deg)_ml']]
    ml.columns = ['ra', 'dec']
    
    ml['idCatalog'] = 2
    
    # concatenating both catalogs to use Nacluster
    final = pd.concat([castles, ml])
    final.reset_index(drop = True, inplace=True)
    final = final.sample(frac=1)  # shuffling dataframe
    
    # saving csv file
    #system_na .to_csv('catalogs.csv', header=False)

    return final

In [13]:
system_na = system_cluster()
system_na

,ra,dec,idCatalog
112,32.372207,-6.719794,2
86,294.604958,66.814500,1
59,203.894958,1.301528,1
34,150.336583,55.897056,1
122,41.642076,-8.426707,2
...,...,...,...
43,157.305625,26.392167,1
105,9.271704,9.163992,2
44,158.392000,7.190417,1
69,216.158708,22.933500,1


In [14]:
# After running NACluster (Java)
# system table  ------------> 138 clustres
def NAC_result():
    old_file = open("clusters_catalogs.csv", 'r+')
    new_file = open("clusters.csv", 'a+')

    lastline=''
    cluster = 0
    header='id,ra,dec,cluster\n'
    new_file.write(header)
    for line in old_file:

        if 'cluster' in line:
            cluster+=1

        elif "Centroid" not in line:
            line = line.replace('(', '')
            line = line.replace(')', '')
            
            newline = ','.join(line.split(',')[0:3]) + ',{}'.format(cluster)
            new_file.write(newline)
            new_file.write('\n')
            
               
    old_file.close()
    new_file.close()
    
    return None
        

In [84]:
NAC_result()

In [89]:
def integrate():
    castle = castles_imgs()
    ml = masterlens_df(50)
    
    nac = pd.read_csv('clusters.csv')  # only has ra, dec, id, cluster
    nac = nac.sort_values(by=['id'])
    nac.reset_index(inplace=True, drop=True)
    
    # first concat nac with castle
    result = pd.concat([nac, castle.add_suffix('_c')], join ='outer', sort=True, axis=1)  #remove axis=1 to drop 'id' columns

    # then concat nac + castle with ml
    integrated = pd.concat([result, ml.add_suffix('_ml')], join ='outer', sort=True, axis=1)
    
    # combining dup rows into one   
    integrated = integrated.groupby('cluster').first()


    integrated = integrated.drop(['id_c', 'id_ml', 'ra', 'dec'], axis=1)
    integrated['isReal'] = 1  #  column to distinguish between real and simulated lenses (real=1, sim=0)
    integrated['Target'] = 1  #  column to distinguish between images with ot without lens (lens=1, nolens=0)

#    system.to_csv("integrated_system.csv", index=False)
    return integrated

In [90]:
integrate()

,id,Lens Name_c,zs_c,zl_c,RA (J2000)_c,Dec (J2000)_c,E(B-V)_c,ms (mag)_c,ml (mag)_c,FGHz (mJy)_c,...,Einstein_R_ml,Einstein_R quality_ml,Stellar_v_disp_ml,Stellar_v_disp_err_ml,Ra(deg)_ml_ml,Dec(deg)_ml_ml,img_path_ml,Alternate Names_ml,isReal,Target
cluster,,,,,,,,,,,,,,,,,,,,,
1.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.500,Reference redshift,NaN,NaN,9.271704,9.163992,['/home/viviane/GravLens/Database/Scraping/mas...,A68,1,1
2.0,11.0,SDSS0246-0825,1.68,NaN,41.642125,-8.426722,0.026,I=16.98/2,I=20.82,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3.0,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,SIE model,NaN,NaN,41.642076,-8.426707,['/home/viviane/GravLens/Database/Scraping/mas...,SDSS J024634.09-082536.1,1,1
4.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,137.862500,5.847780,['/home/viviane/GravLens/Database/Scraping/mas...,NaN,1,1
5.0,38.0,SDSS1004+4112,1.734,0.68,151.145458,41.211889,0.013,i=17.53/4,i=18.42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
6.0,59.0,SDSS1332+0347,1.445,0.191,203.094250,3.794417,0.026,i=18.70/2,i=18.64,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
7.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000,SIE model,NaN,NaN,3.609417,-30.382194,['/home/viviane/GravLens/Database/Scraping/mas...,NaN,1,1
8.0,3.0,HST01247+0352,NaN,NaN,21.185000,3.866667,0.029,I=24.13/2,I=21.86,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
9.0,2.0,HE0047-1756,1.66,0.41,12.615958,-17.669111,0.022,I=16.53/2,I=18.97,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


In [18]:
# Comparing merge by name with NAC

clusters = list(nac["cluster"])
dupes = list(set([x for x in clusters if clusters.count(x) > 1])) # cluster duplicates

clusters_dupes = nac.loc[nac['cluster'].isin(dupes)] 
#clusters_dupes

#merge_name = pd.read_csv('concat.csv')
#problem = [31, 141, 37, 149, 27, 135, 12, 125, 10, 122]
#merge_name.loc[merge_name.index.isin(problem)]

# Merge by name is wrong
# Use merge with NAC

NameError: name 'nac' is not defined

## Lens objects

In [19]:
# Scraping Joao
# lens objects  -  227
castles2 = pd.read_csv('CastelLensData.csv')
castles2 = castles2.set_index('lens_names')

HE_c = castles2.loc[ 'HE0435-1223' , : ]
HE_c

,ra,ra_err,dec,dec_err,description
lens_names,,,,,
HE0435-1223,4.63747,0.000,-12.2873,0.000,deflected
HE0435-1223,3.16147,0.003,-11.7343,0.003,deflected
HE0435-1223,2.17047,0.003,-12.8903,0.005,deflected
HE0435-1223,3.69847,0.003,-13.9013,0.003,deflected
HE0435-1223,3.47247,0.003,-12.8603,0.003,galaxy


In [21]:
# Object table
#HE_ml = pd.read_csv('Scraping/results/redshift_28.csv')
#HE_ml.drop('Unnamed: 1', axis=1, inplace=True)
#HE_ml.rename(columns={"Unnamed: 0": "Description"})

## BLF Challenge Simulations

In [ ]:
blf = pd.read_csv('/home/viviane/BLF_training/image_catalog.csv', comment='#')
blf

## Main code

In [158]:
# Preparing to run NACluster
system_prep = system_cluster()

# run NACluster (Java)
# Saves results in "clusters_catalogs.csv"

# After running NACluster:
# reads "clusters_catalogs.csv"
# Saves result in "clusters.csv"
NAC_result()

# Final system df after integration
# reads "clusters.csv"
# Saves result to "integrated_system.csv"
system_df = integrate()

